In [1]:
# импортируем класс UniversalDetector библиотеки chardet
from chardet.universaldetector import UniversalDetector

# создаем экземпляр класса UniversalDetector
detector = UniversalDetector()

# определяем кодировку файла Credit_OTP.csv
with open('Data/Credit_OTP.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'windows-1251', 'confidence': 0.8897840802899516, 'language': 'Russian'}


In [2]:
# импортируем необходимые библиотеки, классы и модули
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from copy import copy
import warnings
import os

In [3]:
def get_filelen(fname):
    """
    Получает длину csv-файла.
    
    Параметры:
        fname: имя файла.
    Возвращает:
        длина файла.
    """
    cnt_lines = -1
    with open(fname, "rb") as fin:
        for line in fin:
            if len(line.strip()) > 0:
                cnt_lines += 1
                
    return cnt_lines


def get_batch_ids(arr, batch_size):
    """
    Генератор последовательностей, разбитых на батчи.
    
    Параметры:
        arr: последовательность.
        batch_size: размерность.
    Возвращает:
        Батчи.
    """
    n = 0
    while n < len(arr):
        yield arr[n : n + batch_size]
        n += batch_size

        
def read_csv_batch(file, offset, cnt, **read_csv_params):
    """
    Читает батч данных из csv-файла.
    
    Параметры:
        file: путь к файлу.
        offset: отступ.
        cnt: количество строк для чтения.
        **read_csv_params: вспомогательные параметры.
    Возвращает:
        Прочитанные данные.
    """
    read_csv_params = copy(read_csv_params)
    if read_csv_params is None:
        read_csv_params = {}

    try:
        usecols = read_csv_params.pop("usecols")
    except KeyError:
        usecols = None

    header = pd.read_csv(file, nrows=0, **read_csv_params).columns

    with open(file, "rb") as f:
        f.seek(offset)
        data = pd.read_csv(f, header=None, names=header, 
                           chunksize=None, nrows=cnt, 
                           usecols=usecols, **read_csv_params)

    return data


def get_file_offsets(file, n_jobs=None, batch_size=None):
    """
    Получает отступы.
    
    Параметры:
        file: путь к файлу.
        n_jobs: количество ядер процессора для распараллеливания.
        batch_size: размер батча.
    Возвращает:
        Кортеж отступов.
    """ 
    
    assert (
        n_jobs is not None or batch_size is not None
    ), "One of n_jobs or batch size should be defined"

    lens = []
    with open(file, "rb") as f:
        header_len = len(f.readline())
        length = 0
        for row in f:
            if len(row.strip()) > 0:
                lens.append(length)
                length += len(row)

    lens = np.array(lens, dtype=np.int64) + header_len

    if batch_size:
        indexes = list(get_batch_ids(lens, batch_size))
    else:
        indexes = np.array_split(lens, n_jobs)

    offsets = [x[0] for x in indexes]
    cnts = [x.shape[0] for x in indexes]

    return offsets, cnts


def _check_csv_params(**read_csv_params):
    """
    Проверяет параметры для функции `read_csv`.
    
    Параметры:
        **read_csv_params: прочитывает параметры.
    Возвращает:
        Новые параметры.
    """
    for par in ["skiprows", "nrows", "index_col", 
                "header", "names", "chunksize"]:
        if par in read_csv_params:
            read_csv_params.pop(par)
            warnings.warn(
                "Parameter {0} will be ignored in parallel mode".format(par),
                UserWarning,
            )

    return read_csv_params


def read_csv(file, n_jobs=4, **read_csv_params):
    """
    Прочитывает данные из csv-файла.
    
    Параметры:
        file: путь к файлу.
        n_jobs: количество ядер процессора для распараллеливания.
        **read_csv_params: вспомогательные параметры.
    Возвращает:
        Прочитанные данные.
    """
    if n_jobs == 1:
        return pd.read_csv(file, **read_csv_params)

    if n_jobs == -1:
        n_jobs = os.cpu_count()

    _check_csv_params(**read_csv_params)
    offsets, cnts = get_file_offsets(file, n_jobs)

    with Parallel(n_jobs) as p:
        res = p(
            delayed(read_csv_batch)(file, offset=offset, cnt=cnt, 
                                    **read_csv_params)
            for (offset, cnt) in zip(offsets, cnts)
        )

    res = pd.concat(res, ignore_index=True)

    return res

In [4]:
%%time

data = read_csv('Data/paribas_train.csv', n_jobs=1)

CPU times: user 1.07 s, sys: 120 ms, total: 1.19 s
Wall time: 1.19 s


In [5]:
# импортируем библиотеку sqlite3
import sqlite3

# считываем данные о котировках акций из CSV-файла
msft = pd.read_csv('Data/msft.csv')
msft['Symbol']='MSFT'
aapl = pd.read_csv('Data/aapl.csv')
aapl['Symbol']='AAPL'

In [6]:
# создаем подключение
connection = sqlite3.connect('Data/stocks.sqlite')
# .to_sql() создаст базу SQL для хранения датафрейма в указанной таблице.  
# параметр if_exists задает действие, которое нужно выполнить в том случае,
# если таблица уже существует ('fail' - выдать ошибку ValuError, 'replace' -
# удалить таблицу перед вставкой новых значений, 'append' - вставить
# новые значения в существующую таблицу)
msft.to_sql('STOCK_DATA', connection, if_exists='replace')
aapl.to_sql('STOCK_DATA', connection, if_exists='append')

# подтверждаем отправку данных в базу и закрываем подключение
connection.commit()
connection.close()

In [7]:
# подключаемся к файлу базы данных
connection = sqlite3.connect('Data/stocks.sqlite')

# запрос всех записей в STOCK_DATA
# возвращает датафрейм
# index_col задает столбец, который нужно сделать
# индексом датафрейма
stocks = pd.read_sql("SELECT * FROM STOCK_DATA;", 
                     connection, index_col='index')

# закрываем подключение
connection.close()

# выводим первые 5 наблюдений в извлеченных данных
stocks[:5]

,Date,Open,High,Low,Close,Volume,Symbol
index,,,,,,,
0,7/21/2014,83.46,83.53,81.81,81.93,2359300,MSFT
1,7/18/2014,83.30,83.40,82.52,83.35,4020800,MSFT
2,7/17/2014,84.35,84.63,83.33,83.63,1974000,MSFT
3,7/16/2014,83.77,84.91,83.66,84.91,1755600,MSFT
4,7/15/2014,84.30,84.38,83.20,83.58,1874700,MSFT


In [8]:
# открываем подключение
connection = sqlite3.connect('Data/stocks.sqlite')
# создаем строку-запрос
query = "SELECT * FROM STOCK_DATA WHERE Volume>29200100 AND Symbol='MSFT';"
# выполняем запрос
items = pd.read_sql(query, connection, index_col='index')

# выводим результат запроса
items

,Date,Open,High,Low,Close,Volume,Symbol
index,,,,,,,
1081,5/21/2010,42.22,42.35,40.99,42.00,33610800,MSFT
1097,4/29/2010,46.80,46.95,44.65,45.92,47076200,MSFT
1826,6/15/2007,89.80,92.10,89.55,92.04,30656400,MSFT
3455,3/16/2001,47.00,47.80,46.10,45.33,40806400,MSFT
3712,3/17/2000,49.50,50.00,48.29,50.00,50860500,MSFT


In [9]:
# можно воспользоваться функцией pandas.read_sql_query()
items2 = pd.read_sql_query(
    "SELECT * FROM STOCK_DATA WHERE Volume>29200100 AND Symbol='MSFT';", 
    connection, 
    index_col='index')
# закрываем подключение
connection.close()

# выводим результат запроса
items2

,Date,Open,High,Low,Close,Volume,Symbol
index,,,,,,,
1081,5/21/2010,42.22,42.35,40.99,42.00,33610800,MSFT
1097,4/29/2010,46.80,46.95,44.65,45.92,47076200,MSFT
1826,6/15/2007,89.80,92.10,89.55,92.04,30656400,MSFT
3455,3/16/2001,47.00,47.80,46.10,45.33,40806400,MSFT
3712,3/17/2000,49.50,50.00,48.29,50.00,50860500,MSFT
